In [1]:
import pandas as pd

In [2]:
from sklearn.feature_extraction import DictVectorizer

In [3]:
from sklearn.linear_model import LinearRegression


In [4]:
from sklearn.metrics import mean_squared_error
import numpy as np

In [5]:
!pip install pyarrow

  Using cached pyarrow-20.0.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
Using cached pyarrow-20.0.0-cp312-cp312-manylinux_2_28_x86_64.whl (42.3 MB)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [6]:
df_jan = pd.read_parquet('../data/yellow_tripdata_2023-01.parquet')

In [7]:
df_jan.shape[1]

19

In [8]:
df_feb = pd.read_parquet('../data/yellow_tripdata_2023-02.parquet')

In [9]:
df_feb.shape[1]

19

In [10]:
df_jan

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN


In [11]:
df_jan['tpep_pickup_datetime'] = pd.to_datetime(df_jan['tpep_pickup_datetime'])
df_jan['tpep_dropoff_datetime'] = pd.to_datetime(df_jan['tpep_dropoff_datetime'])


In [12]:
df_jan['duration']  = df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']

In [13]:
df_jan['duration']

0         0 days 00:08:26
1         0 days 00:06:19
2         0 days 00:12:45
3         0 days 00:09:37
4         0 days 00:10:50
                ...      
3066761   0 days 00:13:59
3066762   0 days 00:19:27
3066763   0 days 00:24:31
3066764   0 days 00:13:00
3066765   0 days 00:14:24
Name: duration, Length: 3066766, dtype: timedelta64[us]

In [14]:
df_jan['duration_minutes'] = df_jan['duration'].dt.total_seconds() / 60

In [15]:
df_jan['duration_minutes']

0           8.433333
1           6.316667
2          12.750000
3           9.616667
4          10.833333
             ...    
3066761    13.983333
3066762    19.450000
3066763    24.516667
3066764    13.000000
3066765    14.400000
Name: duration_minutes, Length: 3066766, dtype: float64

In [16]:
std = df_jan['duration_minutes'].std()

In [17]:
std

np.float64(42.59435124195458)

In [18]:
df_jan_clean = df_jan[(df_jan['duration_minutes'] >= 1) & (df_jan['duration_minutes'] <=60 )]
print(f"Before: {df_jan.shape[0]} rows")
print(f"After:  {df_jan_clean.shape[0]} rows")


Before: 3066766 rows
After:  3009173 rows


In [19]:
fraction_dropped = 1 - (len(df_jan_clean) / len(df_jan))

In [20]:
print(f"Fraction dropped: {fraction_dropped:.4f}")


Fraction dropped: 0.0188


In [21]:
df_jan_clean.loc[:, 'PULocationID'] = df_jan_clean['PULocationID'].astype(str)
df_jan_clean.loc[:, 'DOLocationID'] = df_jan_clean['DOLocationID'].astype(str)


/tmp/ipykernel_31862/2216521784.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['161' '43' '48' ... '114' '230' '262']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_jan_clean.loc[:, 'PULocationID'] = df_jan_clean['PULocationID'].astype(str)
/tmp/ipykernel_31862/2216521784.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['141' '237' '238' ... '239' '79' '143']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_jan_clean.loc[:, 'DOLocationID'] = df_jan_clean['DOLocationID'].astype(str)


In [22]:
categorical = ['PULocationID', 'DOLocationID']
df_categorical = df_jan_clean[categorical]


In [23]:
dicts = df_categorical.to_dict(orient='records')


In [24]:
dv = DictVectorizer()
X = dv.fit_transform(dicts)

In [25]:
print(f"Shape of X: {X.shape}")
print(f"Example features: {dv.feature_names_[:5]}")


Shape of X: (3009173, 515)
Example features: ['DOLocationID=1', 'DOLocationID=10', 'DOLocationID=100', 'DOLocationID=101', 'DOLocationID=102']


In [26]:
y = df_jan_clean['duration_minutes'].values


In [27]:
model = LinearRegression()
model.fit(X, y)

LinearRegression()

In [28]:
y_pred = model.predict(X)


In [ ]:
rmse = np.sqrt(mean_squared_error(y, y_pred))
print(f"RMSE on training data: {rmse:.2f}")

RMSE on training data: 7.65


: 

In [ ]:
df_feb['duration'] = df_feb['tpep_dropoff_datetime'] - df_feb['tpep_pickup_datetime']
df_feb['duration_minutes'] = df_feb['duration'].dt.total_seconds() / 60

df_feb_clean = df_feb[(df_feb['duration_minutes'] >= 1) & (df_feb['duration_minutes'] <= 60)].copy()

df_feb_clean['PULocationID'] = df_feb_clean['PULocationID'].astype(str)
df_feb_clean['DOLocationID'] = df_feb_clean['DOLocationID'].astype(str)

categorical = ['PULocationID', 'DOLocationID']
dicts_val = df_feb_clean[categorical].to_dict(orient='records')

X_val = dv.transform(dicts_val)

y_val = df_feb_clean['duration_minutes'].values
y_pred_val = model.predict(X_val)

rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
print(f"RMSE on validation data (Feb 2023): {rmse_val:.2f}")